In [1]:
# Load your workspace in a brand new environment
from azureml.core import Workspace
ws = Workspace.from_config(path="./.azureml/ws_config.json")

In [2]:
# import libraries
import fastai
from fastai import *
from fastai.text.all import *

#Import Data
path = untar_data(URLs.IMDB)
dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')

In [3]:
#create a leaner and fine tune
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy) 
learn.fine_tune(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.458950,0.391510,0.825000,07:23


epoch,train_loss,valid_loss,accuracy,time
0,0.278413,0.229457,0.908640,14:45


In [4]:
learn.path = Path(os.getcwd())
learn.export("m.pkl")

In [5]:
#Register Model
from azureml.core.model import Model
model = Model.register(workspace=ws, model_path="./m.pkl", model_name="FastAI")
print(model.name, model.id, model.version, sep='\t')

Registering model FastAI
FastAI	FastAI:2	2


In [4]:
from azureml.core.model import Model
model = Model(ws,"FastAI")

In [5]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
env = Environment(name='myenv')
python_packages = ['fastai', 'torch', 'spacy==2.1.8']
for package in python_packages:
    env.python.conda_dependencies.add_pip_package(package)

inference_config = InferenceConfig(environment=env, source_directory='./', entry_script='./pred.py')

In [6]:
#ACI Deployment

from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 3, memory_gb =1)
service = Model.deploy(ws, "aciservice1", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-02-10 06:28:52+00:00 Creating Container Registry if not exists.
2022-02-10 06:28:52+00:00 Registering the environment.
2022-02-10 06:28:54+00:00 Use the existing image.
2022-02-10 06:28:54+00:00 Generating deployment configuration.
2022-02-10 06:28:55+00:00 Submitting deployment to compute.
2022-02-10 06:29:02+00:00 Checking the status of deployment aciservice1..
2022-02-10 06:30:03+00:00 Checking the status of inference endpoint aciservice1.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
